<a href="https://colab.research.google.com/github/kgautam2103/Zoom_speech2text_AWS_public/blob/main/Zoom_speech2text_AWS_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("mnt",force_remount=True)
%run '/content/mnt/MyDrive/Colab Notebooks/zoom_speech2text_utility_functions.ipynb'
!mkdir ~/.aws
!cp /content/mnt/MyDrive/aws.txt ~/.aws/credentials
!cp /content/mnt/MyDrive/aws_config.txt ~/.aws/config


Mounted at mnt
mkdir: cannot create directory ‘/root/.aws’: File exists


In [ ]:
!pip3 install boto3
!pip3 install pydub

In [ ]:
import requests
import os
import time
from urllib.parse import urlencode
from pydub import AudioSegment
import boto3
import logging
import random
from botocore.exceptions import ClientError
from google.colab import files


meeting_id = input("enter the meeting id like 9181716151 : ")
recording_date = input("enter the recording date like 2021-07-25 : ")
base_url = 'https://api.zoom.us/v2/'
past_meetings = base_url+'/past_meetings/'
get_meeting_url = past_meetings+meeting_id+'/instances'
cwd = os.getcwd()
print(cwd)

#put the auth token for your zoom account
auth_token = ''



authorization = 'Bearer ' + auth_token
auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

meeting_id_response = requests.get(get_meeting_url,headers=auth_headers)

if meeting_id_response.status_code == 200:

  for x in meeting_id_response.json()['meetings']:
    if recording_date in str(x['start_time']):
      print(x)
   

enter the meeting id like 9181716151 : 9181716151
enter the recording date like 2021-07-25 : 2021-08-01
/content
{'uuid': 'mE4aQoZXTIq7NogGXmDexg==', 'start_time': '2021-08-01T10:16:30Z'}
{'uuid': 'oh5z2OBRS6ekx+a0TosN9w==', 'start_time': '2021-08-01T01:16:06Z'}
{'uuid': 'rctMH0vbSdS7pfGlj9y/zw==', 'start_time': '2021-08-01T13:06:36Z'}
{'uuid': 'wHc5q3qBT2+RfFdg8Gt9dg==', 'start_time': '2021-08-01T07:16:38Z'}


In [ ]:
#based on the above result, please enter the meeting uuid
meeting_uuid = input("enter the meeting uuid like wHc5q3qBT2+RfFdg8Gt9dg== : ")
project_id='bigdata-poc-281913'
s3_bucket_name='zoom2021speech2text'
key='/content/adc.json'

double_encoded_meeting_uuid = double_encode_uuid(meeting_uuid)

recording_url='meetings/'+double_encoded_meeting_uuid+'/recordings'
final_recording_url=base_url+recording_url

recording_response = requests.get(final_recording_url,headers=auth_headers)

recording_download_url,start_ts = get_recording_download_url(recording_response.json())
download_response = requests.get(recording_download_url)

download_file_name=start_ts+".m4a"
wav_file_name = start_ts+".wav"
#transcript_file_name = cwd+"/"+start_ts+".txt"

open(download_file_name,'wb').write(download_response.content)
print("m4a written to local")

input_filepath = cwd+"/"+download_file_name
wav_filepath = m4a_to_wav(input_filepath)
print("m4a converted to wav")


is_uploaded = s3_upload_file(wav_file_name,s3_bucket_name)
print("audio file uploaded to s3")

job_name= "job_transcribe_"+wav_file_name.replace(":","")+"_"+str(random.randint(0,99999))
job_uri="s3://"+s3_bucket_name+"/"+wav_file_name
aws_region="ap-south-1"
download_transcribefile_name="transcribe_result_"+start_ts+"_"+str(random.randint(0,99999))+".txt"

if is_uploaded:
  transcribe_response = aws_transcribe(job_name,job_uri,aws_region)
  print(transcribe_response['TranscriptionJob']['TranscriptionJobStatus'])
  if transcribe_response['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
    transcript_uri=transcribe_response['TranscriptionJob']['Transcript']['TranscriptFileUri']
    print(transcript_uri)
    transcript_json = requests.get(transcript_uri)
    transcribe_content=transcript_json.json()['results']['transcripts'][0]['transcript']
    open(download_transcribefile_name,'w+').write(transcribe_content)
    files.download(download_transcribefile_name)
    print(transcribe_content)
    s3_delete_file(s3_bucket_name,wav_file_name)
    
else:
  print("file upload to S3 failed")